Working with Elise to speed up the matching of true halos to observed mock Gaia stars. 

In [3]:
import pandas as pd 
import numpy as np
import h5py 
from itertools import izip
import cPickle as pickle 
from collections import OrderedDict

if mask_dict becomes too large, may need to save several, like above. Then when you switch from one to the next, you need to add the length of the previous to the indexing  to mean_vals

In [17]:
mask_dict = OrderedDict()
#i = 5
for j in xrange(10):
    print 'j',j
    with open('/home/users/swmclau2/scratch/parentid_dict_%d.pkl'%j, 'r') as f:
        parentid_dict = pickle.load(f)
        
    for i in xrange(5, 10): #, 600):
        print 'i',i
        fname = '/oak/stanford/orgs/kipac/users/enadler/m12i_rockstar_dm/catalog_hdf5/star_%03d.hdf5'%i

        with h5py.File(fname, 'r') as f:
            mask = np.array(f['star.number']) != -1
            ar = np.array(f['star.indices'])[mask]
            halo_ID = np.array(f['id'])[mask]

            #for k, stars in enumerate(ar):
            #    key = halo_ID[k]
            #    mask_dict[key] = []
            #    for j in xrange(10):
            #        mask1 = [parentid_dicts[j][s] for s in stars if s in parentid_dicts[j]]
            #        mask_dict[key].append(mask1)


            for k, stars in enumerate(ar):
                print k,
                key = halo_ID[k]
                if key not in mask_dict:
                    mask_dict[key] = [[] for _ in xrange(10)]
                
                mask1 = [parentid_dict[s] for s in stars if s in parentid_dict]
                if len(mask1)>0:
                    mask_dict[key][j].extend(mask1)
            print
            print '*'*10

j 0
i 5
0 1 2 3
**********
i 6
0 1 2 3 4
**********
i 7
0 1 2 3 4 5 6 7 8
**********
i 8
0 1 2 3 4 5 6 7 8
**********
i 9
0 1 2 3 4 5 6 7 8 9 10 11 12
**********
j 1
i 5
0 1 2 3
**********
i 6
0 1 2 3 4
**********
i 7
0 1 2 3 4 5 6 7 8
**********
i 8
0 1 2 3 4 5 6 7 8
**********
i 9
0 1 2 3 4 5 6 7 8 9 10 11 12
**********
j 2
i 5
0 1 2 3
**********
i 6
0 1 2 3 4
**********
i 7
0 1 2 3 4 5 6 7 8
**********
i 8
0 1 2 3 4 5 6 7 8
**********
i 9
0 1 2 3 4 5 6 7 8 9 10 11 12
**********
j 3
i 5
0 1 2 3
**********
i 6
0 1 2 3 4
**********
i 7
0 1 2 3 4 5 6 7 8
**********
i 8
0 1 2 3 4 5 6 7 8
**********
i 9
0 1 2 3 4 5 6 7 8 9 10 11 12
**********
j 4
i 5
0 1 2 3
**********
i 6
0 1 2 3 4
**********
i 7
0 1 2 3 4 5 6 7 8
**********
i 8
0 1 2 3 4 5 6 7 8
**********
i 9
0 1 2 3 4 5 6 7 8 9 10 11 12
**********
j 5
i 5
0 1 2 3
**********
i 6
0 1 2 3 4
**********
i 7
0 1 2 3 4 5 6 7 8
**********
i 8
0 1 2 3 4 5 6 7 8
**********
i 9
0 1 2 3 4 5 6 7 8 9 10 11 12
**********
j 6
i 5
0 1 2 3
**********
i

In [18]:
print len(mask_dict)

36


In [19]:
with open('/home/users/swmclau2/scratch/mask_dict.pkl', 'w') as f:
    pickle.dump(mask_dict, f)

In [20]:
keys= ['ra_true', 'dec_true', 'pmra_true', 'pmdec_true', 'parallax_true', 'b_true']

In [21]:
mean_vals = np.zeros((len(mask_dict.keys()), len(keys)))

In [31]:
for j in range(10):
    fname1 = '/oak/stanford/orgs/kipac/edarragh/m12i_lsr0/slice-%d.hdf5'%j
    print '*'*10
    if not any([v[j] for v in mask_dict.itervalues()]):
        continue # don't load this file if there's nothing in them 
    with h5py.File(fname1, 'r') as f1:
        for k_no, k in enumerate(keys):
            print '~'*10, k_no, '~'*10
            val = f1[k][()] # this is a slow disk read
            
            for obj_no, (obj_key, obj_mask) in enumerate(mask_dict.iteritems()):
                print obj_no,
                if len(obj_mask)==0 or len(obj_mask[j]) == 0:
                    print obj_mask[j]
                    continue
                print obj_no
                #print obj_mask[j]
                #print j, k_no, obj_no
                #mean_vals[obj_no, k_no] = val[obj_mask[j]].mean()
                mean_vals[obj_no, k_no] = np.mean([val[om[0]:om[1]].mean() for om in obj_mask[j]])

            del val
            break

**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
~~~~~~~~~~ 0 ~~~~~~~~~~
0 0
1 1
2 2
3 3
4 []
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 []
13 13
14 14
15 []
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 []
30 30
31 []
32 32
33 33
34 []
35 []
~~~~~~~~~~ 1 ~~~~~~~~~~
0 0
1 1
2 2
3 3
4 []
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 []
13 13
14 14
15 []
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 []
30 30
31 []
32 32
33 33
34 []
35 []
~~~~~~~~~~ 2 ~~~~~~~~~~
0 0
1 1
2 2
3 3
4 []
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 []
13 13
14 14
15 []
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 []
30 30
31 []
32 32
33 33
34 []
35 []
~~~~~~~~~~ 3 ~~~~~~~~~~
0 0
1 1
2 2
3 3
4 []
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 []
13 13
14 14
15 []
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 []
30 30
31 []
32 32
33 33
34 []
35 []
~~~~~~

KeyboardInterrupt: 

In [24]:
len(mask_dict)

36

In [29]:
mean_vals[4]

array([ 0.,  0.,  0.,  0.,  0.,  0.])

In [26]:
np.save('mean_vals.npy', mean_vals)